## Check integrity of designed dataframe and finalize

In [2]:
import pandas as pd

In [ ]:
path_to_synthesis_files='../Data/data_for_synthesis_pipeline/'

In [3]:
chip1=pd.read_csv(f'{path_to_synthesis_files}chip_df.csv',index_col=0)

In [4]:
len(chip1)

252250

In [5]:
chip1.head()

,category,control,rep,aa_seq,barcode,coding,full_nt_seq
0,rnn_designed_plus_rand_train_walked,0,1,DEDEIRTTNPVATEQYGSVSTNLgGEGdNR,GTGTGTCCAGAATGCGGT,GACGAAGATGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGT...,GGGTCACGCGTAGGAGAAGACATTACAGACGAAGATGAAATCAGGA...
1,rnn_designed_plus_rand_train_walked,0,1,DEDEIRTTNPVATEQYGSVSTNLQDnGnNdR,GTCCGTTCTAATATCAGA,GACGAGGACGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGT...,GGGTCACGCGTAGGAGAAGACATTACAGACGAGGACGAAATCAGGA...
2,rnn_designed_plus_rand_train_walked,0,1,DEDEIRTTNPVATEQYGAVSTNLQGdGNdR,GCTAATCACACGTTGCAT,GACGAGGACGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGT...,GGGTCACGCGTAGGAGAAGACATTACAGACGAGGACGAAATCAGGA...
3,rnn_designed_plus_rand_train_walked,0,1,DEEEIATTNPVATEQYGSVSTNLQHdGDeR,GAAGGAACTGGAACTGTA,GACGAAGAGGAAATCGCTACAACCAATCCCGTGGCTACGGAGCAGT...,GGGTCACGCGTAGGAGAAGACATTACAGACGAAGAGGAAATCGCTA...
4,rnn_designed_plus_rand_train_walked,0,1,DEHEIRTTNPVATEQYGNVSTNLQGgGdNR,AGGACAGCTAACCTCAAT,GACGAACACGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGT...,GGGTCACGCGTAGGAGAAGACATTACAGACGAACACGAAATCAGGA...


In [10]:
from collections import Counter

In [11]:
print(Counter(chip1.category))

Counter({'random_doubles': 33320, 'rnn_standard_walked': 21386, 'rnn_designed_plus_rand_train_walked': 21371, 'cnn_designed_plus_rand_train_walked': 21346, 'rnn_rand_doubles_plus_singles_walked': 21287, 'cnn_rand_doubles_plus_single_walked': 21235, 'lr_standard_walked': 21097, 'cnn_standard_walked': 21088, 'lr_rand_doubles_plus_single_walked': 20942, 'lr_designed_plus_rand_train_walked': 20236, 'singles': 4448, 'previous_chip_viable': 3000, 'previous_chip_nonviable': 3000, 'lr_rand_doubles_plus_single_seed': 2096, 'rnn_designed_plus_rand_train_seed': 2088, 'rnn_rand_doubles_plus_singles_seed': 2064, 'lr_designed_plus_rand_train_seed': 2041, 'cnn_rand_doubles_plus_single_seed': 2040, 'lr_standard_seed': 2004, 'cnn_standard_seed': 1940, 'rnn_standard_seed': 1934, 'cnn_designed_plus_rand_train_seed': 1916, 'wild_type': 200, 'stop': 171})


In [12]:
print(Counter(chip1[chip1.control==0].category))

Counter({'random_doubles': 33320, 'rnn_standard_walked': 20636, 'rnn_designed_plus_rand_train_walked': 20624, 'cnn_designed_plus_rand_train_walked': 20602, 'rnn_rand_doubles_plus_singles_walked': 20546, 'cnn_rand_doubles_plus_single_walked': 20491, 'cnn_standard_walked': 20350, 'lr_standard_walked': 20347, 'lr_rand_doubles_plus_single_walked': 20201, 'lr_designed_plus_rand_train_walked': 19507, 'lr_rand_doubles_plus_single_seed': 2096, 'rnn_designed_plus_rand_train_seed': 2088, 'rnn_rand_doubles_plus_singles_seed': 2064, 'lr_designed_plus_rand_train_seed': 2041, 'cnn_rand_doubles_plus_single_seed': 2040, 'lr_standard_seed': 2004, 'cnn_standard_seed': 1940, 'rnn_standard_seed': 1934, 'cnn_designed_plus_rand_train_seed': 1916})


In [13]:
print(Counter(chip1[chip1.control==1].category))

Counter({'singles': 4448, 'previous_chip_viable': 3000, 'previous_chip_nonviable': 3000, 'rnn_standard_walked': 750, 'lr_standard_walked': 750, 'rnn_designed_plus_rand_train_walked': 747, 'cnn_designed_plus_rand_train_walked': 744, 'cnn_rand_doubles_plus_single_walked': 744, 'rnn_rand_doubles_plus_singles_walked': 741, 'lr_rand_doubles_plus_single_walked': 741, 'cnn_standard_walked': 738, 'lr_designed_plus_rand_train_walked': 729, 'wild_type': 200, 'stop': 171})


In [14]:
len(chip1[chip1.control==1])

17503

In [15]:
def test_barcodes(chips):
    barcodes=[]
    desired_len=18
    for chip in chips:
        barcode_lens=set([len(x) for x in chip.barcode])
        assert (len(barcode_lens)==1) and (list(barcode_lens)[0]==desired_len) , "barcodes not the correct size"
        assert len(list(chip.barcode))==len(set(list(chip.barcode))), "duplicate barcodes within a chip"
        barcodes.extend(list(chip.barcode))
    assert len(barcodes)==len(set(barcodes)), "duplicate barcodes between chips"
    print ("All barcode tests passed")
    

In [16]:
test_barcodes([chip1])

All barcode tests passed


In [17]:
from Bio import SeqIO 
from Bio.Data import CodonTable
from Bio.Restriction import BbsI, BsaI, BsmBI, EcoRV, HindIII, XbaI, ScaI,BsrBI
from Bio.Seq import Seq 
from Bio.SeqUtils import MeltingTemp as mt

In [18]:
def test_translation(df):
    try:
        assert len(df["coding"])==len(set(df["coding"])),"non unique coding region"
    except:
        print (len(df[df.duplicated("coding", keep=False)]),"duplicated coding regions")
        print (len(df[(df.control==1)&(df.duplicated("coding", keep=False))]),"of duplicated codings are controls")
    for i, row in df.iterrows():
        nt=row["coding"]
        aa=str(Seq(nt).translate())
        assert row["barcode"] in row["full_nt_seq"], "barcode not paired with full seq"
        assert aa==str(row["aa_seq"]).upper(), "wrong translation"
      
    print ("Barcodes match sequences, all coding regions translate to corresponding peptides")

In [19]:
test_translation(chip1)

1051 duplicated coding regions
991 of duplicated codings are controls
Barcodes match sequences, all coding regions translate to corresponding peptides


In [20]:
re_list = []
for x in [BsaI, HindIII, XbaI, BbsI, BsmBI, EcoRV,BsrBI]:
    re_list.append(x.site)
    rc = str(Seq(x.site).reverse_complement())
    if rc != x.site:
        re_list.append(rc)
homopolymer=[]
for nt in ['A', 'C', 'G', 'T']:
    for n in range(5,6):
        site = ''.join([nt]*n)
        re_list.append(site)
        homopolymer.append(site)
print (re_list)

['GGTCTC', 'GAGACC', 'AAGCTT', 'TCTAGA', 'GAAGAC', 'GTCTTC', 'CGTCTC', 'GAGACG', 'GATATC', 'CCGCTC', 'GAGCGG', 'AAAAA', 'CCCCC', 'GGGGG', 'TTTTT']


In [21]:
import re
def check_re_sites(nt_seq, re_list):
    count = 0
    for site in re_list:
        site_loc = re.finditer(site, nt_seq.upper())
        for loc in site_loc:
            count+=1
    return count


In [22]:
def augment_with_reverse_complements(re_list):
    for enz_site in re_list:
        rc = str(Seq(enz_site).reverse_complement())
        if rc not in re_list:
           re_list.append(rc)
    return re_list

In [23]:
def check_cut_site_counts(df):
    for i,row in df.iterrows():
        seq=row["full_nt_seq"]
        assert check_re_sites(seq,augment_with_reverse_complements([BbsI.site]))==2,  "Wrong count BbsI sites"
        assert check_re_sites(seq,augment_with_reverse_complements([BsaI.site]))==2,  "Wrong count BsaI sites"
        assert check_re_sites(seq,augment_with_reverse_complements([BsrBI.site]))==0,  "Wrong count BsrBI sites"
        assert check_re_sites(seq,augment_with_reverse_complements([EcoRV.site]))==1,  "Wrong count EcoRV sites"
        assert check_re_sites(seq,augment_with_reverse_complements([BsmBI.site]))==0,  "Wrong count BsmbI sites"
        assert check_re_sites(seq,augment_with_reverse_complements([HindIII.site]))==0,  "Wrong count HindIII sites"
        assert check_re_sites(seq,augment_with_reverse_complements([XbaI.site]))==0,  "Wrong count XbaI sites"
    else:
        print ("Correct count for BbsI, BsaI, EcoRV,ScaI")

check_cut_site_counts(chip1)


Correct count for BbsI, BsaI, EcoRV,ScaI


In [24]:
# fill sequences with Ts for agilent products to be of the same size
def append_T(seq,max_len=230):
    new_seq="".join(seq)
    while len(new_seq)<max_len:
        new_seq=new_seq+"t"
    return new_seq

#produce a final textfile for ordering
def make_outfile_from_chip(outfile_name,chip,skip=[], max_size=244000,capitalize=False):
    with open(outfile_name,"w") as outfile:
        for i, row in chip.iterrows():
            if i not in skip:
                out_seq=""
                if len(row.full_nt_seq)<230:
                    out_seq=append_T(row.full_nt_seq)
                else:
                    out_seq=row.full_nt_seq
                if capitalize:
                   out_seq=out_seq.upper() 
                outfile.write(out_seq+"\n")
            if i==(max_size+len(skip)-1):
                break


In [26]:
# BsrBI=Skipping variant 3824 because of primer annealing on the sequence:  DEHEIATTNPVAYEQWGSVSTNLQSgGeNdD	

#make_outfile_from_chip(f'{path_to_synthesis_files}chip_for_agilent.txt',chip1, skip=[3824],capitalize=True)

In [27]:
chip1[chip1.category=="wild_type"].iloc[0].full_nt_seq # chip1 wt in the final file at line 218730

'GGGTCACGCGTAGGAGAAGACATTACAGACGAAGAGGAAATCAGGACAACCAATCCCGTGGCTACGGAGCAGTATGGTTCTGTATCTACCAACCTCCAGAGAGGCAACAGACAAGCGAGACCGATATCGGTCTCACGCTGTAATGCGGTCTGAGCCGCGCTAAGTCTTCGTGTGGCTGCGGAAC'